In [69]:
import numpy as np
import pandas as pd
import math
import csv

import sklearn

from sklearn.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import GradientBoostingClassifier
# import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBClassifier

In [57]:
data = pd.read_csv('data/cs-training.csv')
test = pd.read_csv('data/cs-test.csv')
y_test = pd.read_csv('data/sampleEntry.csv')

features = pd.read_excel('data/Data Dictionary.xls')

# x_train = np.loadtxt('data/cs-training.csv', skiprows=1, delimiter=',')
# x_test = np.loadtxt('data/cs-test.csv', skiprows=1, delimiter=',')
data

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [3]:
features

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Variable Name,Description,Type
1,SeriousDlqin2yrs,Person experienced 90 days past due delinquenc...,Y/N
2,RevolvingUtilizationOfUnsecuredLines,Total balance on credit cards and personal lin...,percentage
3,age,Age of borrower in years,integer
4,NumberOfTime30-59DaysPastDueNotWorse,Number of times borrower has been 30-59 days p...,integer
5,DebtRatio,"Monthly debt payments, alimony,living costs di...",percentage
6,MonthlyIncome,Monthly income,real
7,NumberOfOpenCreditLinesAndLoans,Number of Open loans (installment like car loa...,integer
8,NumberOfTimes90DaysLate,Number of times borrower has been 90 days or m...,integer
9,NumberRealEstateLoansOrLines,Number of mortgage and real estate loans inclu...,integer


In [4]:
data['MonthlyIncome'] = data['MonthlyIncome'].fillna(data['MonthlyIncome'].mean(axis=0))
data['NumberOfDependents'] = data['NumberOfDependents'].fillna(1)

In [5]:
data.isnull().any(axis=0)

Unnamed: 0                              False
SeriousDlqin2yrs                        False
RevolvingUtilizationOfUnsecuredLines    False
age                                     False
NumberOfTime30-59DaysPastDueNotWorse    False
DebtRatio                               False
MonthlyIncome                           False
NumberOfOpenCreditLinesAndLoans         False
NumberOfTimes90DaysLate                 False
NumberRealEstateLoansOrLines            False
NumberOfTime60-89DaysPastDueNotWorse    False
NumberOfDependents                      False
dtype: bool

In [6]:
# 
y = data.loc[:,['SeriousDlqin2yrs']]
x = data.drop(['Unnamed: 0','SeriousDlqin2yrs'],axis=1, inplace=False)

In [7]:
y = np.array(y).reshape(150000)
x = np.array(x)

In [32]:
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2, random_state=2021)

In [33]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((120000, 10), (120000,), (30000, 10), (30000,))

In [34]:
# KNN
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
score = knn.score(x_val, y_val)
score

0.9318333333333333

In [35]:
# Logistic Regression
LR = LogisticRegression()
LR.fit(x_train, y_train)
score = LR.score(x_val, y_val)
score

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9328

In [36]:
# Simple Decision Tree
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train, y_train)
score = model_tree.score(x_val, y_val)
score

0.8950333333333333

In [37]:
# Random Forest
forest = RandomForestClassifier()
forest.fit(x_train, y_train)
score = forest.score(x_val, y_val)
score

0.9347333333333333

In [38]:
# GBDT
gbm = GradientBoostingClassifier()
gbm.fit(x_train, y_train)
score = gbm.score(x_val, y_val)
score

0.9361

In [39]:
# LightGBM
lgb = LGBMRegressor()
lgb.fit(x_train, y_train)
score = lgb.score(x_val, y_val)
score

0.2075797972563871

In [40]:
# XGBoost
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
score = xgb.score(x_val, y_val)
score

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:21:57] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9351666666666667

In [63]:
test['MonthlyIncome'] = test['MonthlyIncome'].fillna(test['MonthlyIncome'].mean(axis=0))
test['NumberOfDependents'] = test['NumberOfDependents'].fillna(1)
x_test = test.drop(['Unnamed: 0','SeriousDlqin2yrs'],axis=1, inplace=False)
x_test = np.array(x_test)

In [73]:
y_pred = xgb.predict_proba(x_test)[:,1]

In [64]:
y_test

array([0.08080665, 0.04071874, 0.01196834, ..., 0.00696991, 0.12199356,
       0.04424841])

In [74]:
# 1. 创建文件对象
f = open('y_predict.csv','w',encoding='utf-8')

# 2. 基于文件对象构建 csv写入对象
csv_writer = csv.writer(f)

# 3. 构建列表头
csv_writer.writerow(["Id","Probability"])
for i in range(y_pred.shape[0]):
    csv_writer.writerow([(i+1), y_pred[i]])

f.close()